In [ ]:
%load_ext autoreload
%autoreload 2

import json

import requests

# Path to the credentials file
CREDS_PATH = "/home/yannick/code/plugin-blob-proxy/.secrets/1739367059_blobprox.json"


# Read the credentials
def read_credentials(path: str) -> dict:
    """Read credentials from the specified JSON file

    Args:
        path (str): Path to the credentials JSON file

    Returns:
        dict: Credentials dictionary
    """
    with open(path) as f:
        return json.load(f)


# root_path = "/data"
root_path = "/home/yannick/code/plugin-blob-proxy/test_data"
creds = read_credentials(CREDS_PATH)
file_path = "/home/yannick/code/plugin-blob-proxy/test_data/test.txt"
destination_path = f"{root_path}/test_new.txt"
destination_file = "test_new.txt"
base_url = "http://localhost:8082"
source_path = f"{root_path}/test.txt"
source_file = "test.txt"
# headers containing the decryption key, salt and comparison key for blob proxy authentication
headers = {
    "x-decryption-key": creds["decryption_key"],
    "x-salt": creds["salt"],
    "x-comparison-key": creds["key"],
}

## Request local public RSA key from blob proxy

In [ ]:
response = requests.get(f"{base_url}/key/public", headers=headers)

response.raise_for_status()
key = response.json()["public_key"]
key

## Send blob to proxy

In [ ]:
from io import BytesIO

import numpy as np
import polars as pl
import requests

blobcontainer = "plugin-aioc-central"
blobname = "test_run/test_7.parquet"
# num_rows = 30_000_000
num_rows = 10_000

print(f"Generating DataFrame with {num_rows:,} rows...")
df = pl.DataFrame(
    {
        "a": np.random.rand(num_rows),
        "b": np.random.rand(num_rows),
        "c": np.random.rand(num_rows),
        "d": np.random.rand(num_rows),
    }
)

# Write the DataFrame to an in‑memory Parquet file
buffer = BytesIO()
df.write_parquet(buffer)
buffer.seek(0)

# Prepare headers for authentication
headers = {"x-decryption-key": creds["decryption_key"], "x-salt": creds["salt"], "x-comparison-key": creds["key"]}

# Prepare form data with public keys, and the file to upload
data = {"public-keys": json.dumps({"default": key})}
files = {"file": buffer}

try:
    response = requests.post(f"{base_url}/upload/{blobcontainer}/{blobname}", headers=headers, files=files, data=data)
    # Close the buffer as soon as the request is sent
    buffer.close()

    # Raise an exception if the request returned an error status (like 409)
    response.raise_for_status()

    # Parse JSON response if successful
    result = response.json()
    print("Upload successful. Result:")
    print(result)

    decryption_key = result["decryption_keys"]["default"]
    print("Decryption key:", decryption_key)

except requests.HTTPError as e:
    print("HTTP error occurred:", e.response.status_code)
    try:
        error_detail = e.response.json()
        print("Error detail:", error_detail)
    except Exception:
        print("Error response content:", e.response.text)
    # Optionally re-raise or handle the error as needed

In [ ]:
blobcontainer = "plugin-aioc-central"
blobname = "test_run/test_7.parquet"


# Prepare headers for authentication
headers = {"x-decryption-key": creds["decryption_key"], "x-salt": creds["salt"], "x-comparison-key": creds["key"]}
data = {"decryption-key": decryption_key}

# Send POST request to upload endpoint
response = requests.get(f"{base_url}/download/{blobcontainer}/{blobname}", headers=headers, data=data)

# Raise an exception for bad responses
response.raise_for_status()

buffer = BytesIO(response.content)
df = pl.read_parquet(buffer)
df